In [ ]:
import librosa
import glob
import os
import numpy as np
import soundfile as sf
from pathlib import Path

In [ ]:
INPUT_DIR = '../media_files/original_audio/moisesdb/moisesdb/moisesdb_v0.1/'
VOCAL_OUTPUT_DIR = '../media_files/separated_audio/code_testing/vocals/'
INSTRUMENTAL_OUTPUT_DIR = '../media_files/separated_audio/code_testing/instrumentals/'

In [ ]:
def merge_vocal_files(input_subdir, target_sr=44100):
    # Lấy danh sách các file .wav trong thư mục con (vocals hoặc instrumentals)
    file_paths = glob.glob(os.path.join(input_subdir, '*.wav'))
    
    if not file_paths:
        print(f"Không tìm thấy file nào trong {input_subdir}")
        return None

    combined_data = []
    
    for file in file_paths:
        # Load audio với sample rate 44.1kHz theo bài báo 
        # mono=True để đảm bảo tín hiệu đơn kênh 
        data, _ = librosa.load(file, sr=target_sr, mono=True)
        
        # Bước quan trọng: Chuẩn hóa đỉnh từng nguồn trước khi trộn [cite: 50]
        data = librosa.util.normalize(data)
        combined_data.append(data)

    # Đảm bảo tất cả các track có cùng độ dài bằng cách bù zero (padding) hoặc cắt ngắn
    max_len = max(len(d) for d in combined_data)
    padded_data = [np.pad(d, (0, max_len - len(d))) for d in combined_data]
    
    # Cộng tuyến tính các tín hiệu đã chuẩn hóa [cite: 50]
    vocal_mix = np.sum(padded_data, axis=0)
    
    # Chuẩn hóa đỉnh một lần nữa sau khi trộn để tránh hiện tượng méo tiếng (clipping) 
    vocal_mix = librosa.util.normalize(vocal_mix)

    return vocal_mix
    

In [ ]:
def merge_instrumental_files(input_subdir, target_sr=44100):

    folder_paths = [
        os.path.join(input_subdir, f) 
        for f in os.listdir(input_subdir) 
        if os.path.isdir(os.path.join(input_subdir, f)) and f.lower() != 'vocals'
    ]
    
    combined_data = []
    
    # 2. Quét qua từng thư mục nhạc cụ (guitar, bass, drums, v.v.)
    for folder in folder_paths:
        # Lấy tất cả file .wav trong thư mục nhạc cụ đó
        file_paths = glob.glob(os.path.join(folder, '*.wav'))
        
        for file in file_paths:
            # Load audio dạng đơn kênh (monaural) với SR=44.1kHz [cite: 51, 53]
            data, _ = librosa.load(file, sr=target_sr, mono=True)
            
            # Chuẩn hóa đỉnh cho từng nguồn trước khi trộn [cite: 50]
            data = librosa.util.normalize(data)
            combined_data.append(data)

    if not combined_data:
        print(f"Không tìm thấy dữ liệu nhạc cụ trong {input_subdir}")
        return None

    # 3. Đảm bảo các track có cùng độ dài (padding zeros)
    max_len = max(len(d) for d in combined_data)
    padded_data = [np.pad(d, (0, max_len - len(d))) for d in combined_data]
    
    # 4. Cộng tuyến tính để tạo non-vocal mixture [cite: 50]
    instrumental_mix = np.sum(padded_data, axis=0)
    
    # 5. Chuẩn hóa đỉnh bản mix cuối cùng 
    instrumental_mix = librosa.util.normalize(instrumental_mix)
    
    return instrumental_mix

In [ ]:
def write_file_to_output(vocal_mix, instrumental_mix, uuid, target_sr=44100):
    # Tạo thư mục đầu ra nếu chưa có
    os.makedirs(VOCAL_OUTPUT_DIR, exist_ok=True)
    os.makedirs(INSTRUMENTAL_OUTPUT_DIR, exist_ok=True)

    # Tạo đường dẫn file hoàn chỉnh: folder/uuid.wav
    vocal_output_path = os.path.join(VOCAL_OUTPUT_DIR, f"{uuid}.wav")
    instrumental_output_path = os.path.join(INSTRUMENTAL_OUTPUT_DIR, f"{uuid}.wav")
    
    # Ghi file bằng soundfile (đảm bảo dữ liệu là mảng float32)
    if vocal_mix is not None:
        sf.write(vocal_output_path, vocal_mix, target_sr)
        
    if instrumental_mix is not None:
        sf.write(instrumental_output_path, instrumental_mix, target_sr)
    
    print(f"--- Đã lưu UUID: {uuid} ---")

In [ ]:
input_path = Path(INPUT_DIR)
subdirs = [d for d in input_path.iterdir() if d.is_dir()]

i = 0

for subdir in subdirs:
    if i == 1:
        break
    uuid = subdir.name
    vocal_mix = merge_vocal_files(subdir / 'vocals')
    instrumental_mix = merge_instrumental_files(subdir)
    write_file_to_output(vocal_mix, instrumental_mix, uuid)
    i+=1